In [46]:
import os
import shutil

In [47]:
i = 0

In [48]:
path = "dataset/preprocessed_images/"
if not os.path.exists(path + "/images"):
    os.makedirs(path + "/images")
if not os.path.exists(path + "/labels"):
    os.makedirs(path + "/labels")

In [49]:
def print_directory_contents(path): 
    global i
    # сортируем директории и файлы для вывода в алфавитном порядке
    sorted_files = sorted(os.listdir(path), key=str.lower)
    for child in sorted_files: 
        child_path = os.path.join(path, child) 
        # поддиректория внутри директории
        if os.path.isdir(child_path): 
            print(f"{i}: {child}") 
            i += 1
            print_directory_contents(child_path) 

In [50]:
def make_dat(path2):
    path = "dataset/preprocessed_images/"
    if not os.path.exists(path + "/images/" + path2):
        os.makedirs(path + "/images/" + path2)
    if not os.path.exists(path + "/labels/" + path2):
        os.makedirs(path + "/labels/" + path2)
    files = os.listdir(path + path2)
    for d in files:
        fc = 0
        for f in os.listdir(path + path2 + d):
            fc += 1
            shutil.copy2(path + path2 + d + "/" + f, path + "images/" + path2 + f)
            with open(path + "/labels/" + path2 + f.replace(".jpg", ".txt"), "w+") as file:
                cl = d.replace("-", "")
                cl = cl.replace("_", "")
                if cl == "999":
                    cl = "998"
                else:
                    if cl == "998":
                        cl = "999"
                file.write(cl + " 0.0 0.0 1.0 1.0")

In [51]:
make_dat("test/")

In [52]:
make_dat("train/")
make_dat("valid/")

1. Функция обучения модели

In [8]:
from ultralytics import YOLO

In [9]:
import yaml

In [36]:
import dill
import time
from datetime import datetime

In [10]:
with open('yolo.yaml', 'r') as file:
    data = yaml.safe_load(file)
data['path'] = os.getcwd() + "/dataset/preprocessed_images/" 

In [11]:
with open('yolo.yaml', 'w') as file:
    yaml.dump(data, file)

In [12]:
data = {
    "epochs" : 1
}

In [41]:
def fun(m, **kwargs):
    models = ["yolo11n.pt", "yolo11m.pt", "yolo11l.pt"]
    index = m
    if index > 2:
        index = 2
    if index < 0:
        index = 0
    epochs = kwargs.get('epochs', 10)
    model = YOLO(models[index])
    start_time = time.time()
    results = model.train(data="yolo.yaml", epochs=epochs)
    end_time = time.time()
    time = end_time - start_time
    etime = time / epochs

    pt_file_path = str(res.save_dir) + "/weights/best.pt" 

    # Укажите директорию и имя для сохранения файла в формате dill
    save_directory = "res"
    now = datetime.now()

    file_name = now

    # Открываем файл для записи (режим 'w' создаст файл, если он не существует)
    with open("res/" + file_name, 'w', encoding='utf-8') as file:
    # Записываем несколько строк в файл
        file.write(f"Время обучения {time / 60}\n")
        file.write(f"Время обучения на эпоху {etime}\n")
        file.write(f"F1_score {res.box.f1}\n")

    # Форматируем время в нужный формат
    current_time = now.strftime("%H_%M_%S")
    save_filename = current_time + ".dill"

    # Убедитесь, что директория для сохранения существует
    os.makedirs(save_directory, exist_ok=True)

    # Полный путь для сохранения файла
    dill_file_path = os.path.join(save_directory, save_filename)

    # Чтение содержимого .pt файла
    with open(pt_file_path, "rb") as pt_file:
        model_data = pt_file.read()

    # Сохранение содержимого в формате dill
    with open(dill_file_path, "wb") as dill_file:
        dill.dump(model_data, dill_file)
    return dill_file_path, res.box.f1, etime, time / 60

In [19]:
fun(0, **data)

Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=yolo.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

train: Scanning /home/c9/Рабочий стол/C9_M3/dataset/preprocessed_images/labels/train.cache... 1000 images, 0 b
val: Scanning /home/c9/Рабочий стол/C9_M3/dataset/preprocessed_images/labels/valid.cache... 1000 images, 0 bac

Plotting labels to runs/detect/train18/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=1e-05, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
MLflow: logging run_id(54db62a59cf946aab8eef561a104b253) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train18
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      3.862      7.805      3.929         12        640: 100%|██████████| 63/63 [09:07<0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/

                   all       1000       1000          0          0          0          0



1 epochs completed in 0.398 hours.
Optimizer stripped from runs/detect/train18/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train18/weights/best.pt, 6.2MB

Validating runs/detect/train18/weights/best.pt...
Ultralytics 8.3.70 🚀 Python-3.11.2 torch-2.6.0+cu124 CPU (13th Gen Intel Core(TM) i5-13400)
YOLO11n summary (fused): 238 layers, 2,920,288 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/

                   all       1000       1000          0          0          0          0


Speed: 2.1ms preprocess, 141.1ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/detect/train18
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
Стандартные оценки на валидационной выборке
Средняя точность (mAP): 0.0
Средняя точность при IoU=0.50: 0.0
F1 Score: []
Precision: []
Recall: []


In [45]:
res.box.f1

[]